In [1]:
# Clear memory
%reset -f

In [3]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import ipywidgets
ipywidgets.Widget.close_all()

import SimpleITK
import datetime
import gc # Garbage collected
import numpy as np
import sys, os
import time
import tifffile
import matplotlib.pyplot as plt
import scipy
from tkinter import Tk
from tkinter.filedialog import askopenfilename, askopenfilenames, askdirectory
import torch
import skimage
from skimage import measure
from skimage.morphology import skeletonize
import pickle
import networkx as nx
import sknw # https://github.com/Image-Py/sknw

import multiprocessing

this_path = os.getcwd()
print(this_path)


import h5py
# import File
from scripts import *
import wrapper_ASTRA

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/global/u2/e/eboigne/tomoTools_als832/tomoTools


In [4]:
path_save = '/global/homes/e/eboigne/cfs_als/2022_wood/'

# case = 'run21_oak_lowHeat'
case = 'run23_walnut_lowHeat'
# case = 'run24_birch_lowHeat'
# case = 'run26_birch_highHeat'

voxel_size = 3.24*2 # [microns]

list_cases_folder_name = sorted([e for e in os.listdir(path_save+case) if not 'probe' in e and not '.tif' in e and not '.pickle' in e])
run = case[3:5]+'_Sample'
list_cases_h5 = sorted([e for e in os.listdir(path_save) if '.h5' in e and run in e])

print(list_cases_folder_name)

['01', '02', '03', '04a', '04b', '05a', '05b', '06a', '06b', '07a', '07b', '08a', '08b', '09a', '09b', '10a', '10b', '11', '12', '13a', '13b', '14a', '14b', '15a', '15b', '16a', '16b', '17a', '17b', '18a', '18b', '19a', '19b', '20a', '20b', '21a', '21b', '22', '23', '24']


In [5]:
ind_slice = 250

slices = []
for ind_case, this_case in enumerate(list_cases_folder_name[:]):
    slices.append(File(path_save+case+'/'+this_case+'/b_movingRegisteredToStatic/').read(ind_slice))
    print(ind_case, this_case)

slices = np.array(slices)
print(slices.shape)
File(path_save+case+'/probe_slice_'+str(ind_slice).zfill(4)).saveTiffStack(slices)

0 01
1 02
2 03
3 04a
4 04b
5 05a
6 05b
7 06a
8 06b
9 07a
10 07b
11 08a
12 08b
13 09a
14 09b
15 10a
16 10b
17 11
18 12
19 13a
20 13b
21 14a
22 14b
23 15a
24 15b
25 16a
26 16b
27 17a
28 17b
29 18a
30 18b
31 19a
32 19b
33 20a
34 20b
35 21a
36 21b
37 22
38 23
39 24
(40, 1280, 1280)


In [5]:
c = []
for this_case in enumerate(list_cases_h5):
    c.append(this_case[1][35:37])

t = []

list_scans = []

for this_case in enumerate(c):
    if str(this_case[1]) in list_cases_folder_name or '-post' in [e for e in list_cases_folder_name if this_case[1] in e][0]:
        cc = list_cases_h5[this_case[0]][9:15]
        h = cc[0:2]
        m = cc[2:4]
        s = cc[4:]
        ts = float(int(h)*3600+int(m)*60+int(s))
        t.append(ts) #c
        print(len(t), this_case, 'c', ts)
    else:
        cc = list_cases_h5[this_case[0]][9:15]
        h = cc[0:2]
        m = cc[2:4]
        s = cc[4:]
        ts0 = float(int(h)*3600+int(m)*60+int(s))
        t.append(ts0) #a
        ts1 = 26.26 + ts0
        t.append(ts1) #b
        print(len(t), this_case, 'a/b', ts0 ,ts1)

scan_times = np.array(t) - t[0]


for ind_scan,scan_time in enumerate(scan_times):
    this_scan = Data()
    # print(ind_scan)
    this_scan.scan_time = scan_time
    this_scan.folder_name = list_cases_folder_name[ind_scan]
    this_scan.full_path = path_save+case+'/'+this_scan.folder_name
    this_scan.ind = ind_scan
    list_scans.append(this_scan)

assert(len(list_cases_folder_name) ==len(scan_times)), 'Wrong sizes'

1 (0, '01') c 56073.0
2 (1, '02') c 56373.0
3 (2, '03') c 56695.0
5 (3, '04') a/b 57252.0 57278.26
7 (4, '05') a/b 57448.0 57474.26
9 (5, '06') a/b 57629.0 57655.26
11 (6, '07') a/b 57811.0 57837.26
13 (7, '08') a/b 57997.0 58023.26
15 (8, '09') a/b 58182.0 58208.26
17 (9, '10') a/b 58369.0 58395.26
18 (10, '11') c 58550.0
19 (11, '12') c 58732.0
21 (12, '13') a/b 58916.0 58942.26
23 (13, '14') a/b 59100.0 59126.26
25 (14, '15') a/b 59282.0 59308.26
27 (15, '16') a/b 59467.0 59493.26
29 (16, '17') a/b 59653.0 59679.26
31 (17, '18') a/b 59838.0 59864.26
33 (18, '19') a/b 60027.0 60053.26
35 (19, '20') a/b 60220.0 60246.26
37 (20, '21') a/b 60403.0 60429.26
38 (21, '22') c 60585.0
39 (22, '23') c 60889.0
40 (23, '24') c 61198.0


In [6]:
print(list_cases_folder_name)

['01', '02', '03', '04a', '04b', '05a', '05b', '06a', '06b', '07a', '07b', '08a', '08b', '09a', '09b', '10a', '10b', '11', '12', '13a', '13b', '14a', '14b', '15a', '15b', '16a', '16b', '17a', '17b', '18a', '18b', '19a', '19b', '20a', '20b', '21a', '21b', '22', '23', '24']


In [7]:
def compute_height(properties):
    for prop in properties:
        bbox = prop.bbox
        prop.height = bbox[3]-bbox[0] # Along z, in pixels
    return(properties)

def compute_equivalent_cylinder_diameters(properties):
    for prop in properties:
        equivalent_cylinder_cross_section = prop.area / prop.height
        prop.equivalent_cylinder_diameter = np.sqrt(4*equivalent_cylinder_cross_section/np.pi) * voxel_size
    return(properties)

def get_path_length(graph, path):
    this_sum=0
    for i in range(len(path)-1):
        this_sum += graph.get_edge_data(path[i], path[i+1])['weight']
    return(this_sum)

def longest_simple_path(graph, source, target):
    longest_path = None
    longest_path_length = 0
    for path in nx.all_simple_paths(graph, source=source, target=target):
        path_length = get_path_length(graph,path)
        if path_length > longest_path_length:
            longest_path_length = path_length
            longest_path = path
    return longest_path, longest_path_length

def find_longest_subgraph_from_ends(graph, bbox):

    ind_end_nodes = [x for x in graph.nodes if graph.degree(x) == 1]
    ind_end_bottom = [x for x in ind_end_nodes if graph.nodes[x]['o'][0] - bbox[0] <=  bbox[3] - graph.nodes[x]['o'][0]]
    ind_end_top = [x for x in ind_end_nodes if not x in ind_end_bottom]

    longest_path_subgraph = None
    longest_path_length = 0
    longest_path = []
    for ind_bot in ind_end_bottom:
        for ind_top in ind_end_top:
            this_longest_path, this_longest_path_length = longest_simple_path(graph, source=ind_bot, target=ind_top)
            this_longest_path_subgraph = graph.subgraph(this_longest_path)

            if this_longest_path_length > longest_path_length:
                longest_path_length = this_longest_path_length
                longest_path = this_longest_path
                longest_path_subgraph = this_longest_path_subgraph

    return(longest_path_subgraph, longest_path, longest_path_length)

def compute_tortuosity(skeleton_properties):

    for ind_pore in range(len(skeleton_properties)):
        bbox = skeleton_properties[ind_pore].bbox
        graph = sknw.build_sknw(skeleton_properties[ind_pore].image, multi=False, iso=False, ring=False, full=True)
        longest_path_subgraph, longest_path, longest_path_length = find_longest_subgraph_from_ends(graph, bbox)

        if longest_path_length == 0:
            longest_path_length = np.nan

        skeleton_properties[ind_pore].length_skeleton = longest_path_length
        skeleton_properties[ind_pore].longest_path = longest_path
        # skeleton_properties[ind_pore].longest_path_subgraph = longest_path_subgraph
        skeleton_properties[ind_pore].tortuosity = longest_path_length / skeleton_properties[ind_pore].height

        # print(ind_pore, longest_path_length, skeleton_properties[ind_pore].tortuosity)
    return(skeleton_properties)

def compute_skeleton(this_scan, mask_pores, force_redo = False):

    if force_redo or not 'b_movingRegisteredToStatic_maskPores_skeleton' in os.listdir(this_scan.full_path):
        skeleton = skeletonize(mask_pores)
        File(this_scan.full_path+'/b_movingRegisteredToStatic_maskPores_skeleton').saveTiffStack(skeleton, type = 'bool')
    else:
        print('\tRe-loading skeleton')
        skeleton = File(this_scan.full_path+'/b_movingRegisteredToStatic_maskPores_skeleton').readAll()
    return(skeleton)

def find_top_bottom_slices_ind(this_scan, mask_solid, ind_all_top, ind_all_bottom):

    this_scan.ind_all_margin = 100
    this_scan.ind_margin = 30
    this_scan.relative_porosity_threshold = 0.15

    mask_solid_slice_sum = np.sum(np.sum(mask_solid, axis = 2), axis = 1)
    mean_area_ind_all = np.mean(mask_solid_slice_sum[ind_all_top+this_scan.ind_all_margin:ind_all_bottom-this_scan.ind_all_margin])

    this_scan.relative_porosity = np.abs(mean_area_ind_all-mask_solid_slice_sum)/mean_area_ind_all

    this_scan.ind_top = np.min(np.where(this_scan.relative_porosity<this_scan.relative_porosity_threshold)) + this_scan.ind_margin
    this_scan.ind_bottom = np.max(np.where(this_scan.relative_porosity<this_scan.relative_porosity_threshold)) - this_scan.ind_margin

    this_scan.height_ROI = this_scan.ind_bottom-this_scan.ind_top + 1

    print('\t\tDetected ROI:\tind_top: '+str(this_scan.ind_top)+', ind_bottom: '+str(this_scan.ind_bottom)+', height: '+str(this_scan.height_ROI))
    if (this_scan.height_ROI < 200):
        print('\t\t\t\t/!\ WARNING: small height of ROI detected: check the cropping of mask solid')

    return(this_scan)

def mask_crop_ROI(mask, ind_top, ind_bottom):
    mask[:ind_top] = False
    mask[ind_bottom:] = False
    return(mask)

def close_mask_solid(this_scan, mask_solid, mask_needle, force_redo = False):

    if force_redo or not 'b_movingRegisteredToStatic_maskSolidClosed' in os.listdir(this_scan.full_path):

        # Option #1: Morphological closing to close the pores
        # chunk_size = (500, 500, 500)
        # filter_half_width = 12
        # structure = custom_3d_kernel_sphere(filter_half_width)
        # mask_solid_dilated = apply_3d_image_processing_on_subvolumes(mask_solid, fast_pytorch_mask_dilation, chunk_size_max = chunk_size, overlap = filter_half_width, radius = filter_half_width)
        # mask_solid_closed = ~apply_3d_image_processing_on_subvolumes(~mask_solid_dilated, fast_pytorch_mask_dilation, chunk_size_max = chunk_size, overlap = filter_half_width, radius = filter_half_width)

        # Option #2: Fill holes algorithm run slice by slice
        n_threads = 16
        mask_solid_closed = np.zeros_like(mask_solid)
        with multiprocessing.Pool(n_threads) as p:
            mask_solid_closed = np.array(p.map(scipy.ndimage.morphology.binary_fill_holes, mask_solid))

        # 1 core version of option #2
        # for ind, slice in enumerate(mask_solid):
        #     mask_solid_closed[ind] = scipy.ndimage.morphology.binary_fill_holes(slice)
        #     print(ind)

        mask_solid_closed[mask_needle] = False
        File(this_scan.full_path+'/b_movingRegisteredToStatic_maskSolidClosed').saveTiffStack(mask_solid_closed, type = 'bool')
    else:
        print('\tRe-loading mask solid closed')
        mask_solid_closed = File(this_scan.full_path+'/b_movingRegisteredToStatic_maskSolidClosed').readAll()

    return(mask_solid_closed.astype('bool'))

def rotate_data(data, rot_coronal_Fiji, rot_sagittal_Fiji, rot_z_Fiji = 0.0):
    pad = 1
    transform_rotation = sitk.Euler3DTransform()
    center_xyz = (data.shape[0] / 2.0+pad, data.shape[1] / 2.0+pad, data.shape[2] / 2.0+pad) # TODO: Is this correct? Not consistent with below
    center_angle = 0.0
    offsets_xyz = (0.0, 0.0, 0.0)

    transform_rotation.SetFixedParameters((center_xyz[0], center_xyz[1], center_xyz[2], center_angle))
    eulerAngle_xyz_deg = (-rot_sagittal_Fiji, rot_coronal_Fiji, 0.0)
    transform_rotation.SetParameters((eulerAngle_xyz_deg[0]/180.0*np.pi, eulerAngle_xyz_deg[1]/180.0*np.pi, eulerAngle_xyz_deg[2]/180.0*np.pi, offsets_xyz[0], offsets_xyz[1], offsets_xyz[2]))

    data_bigger = np.zeros([data.shape[0]+pad*2, data.shape[1]+pad*2, data.shape[2]+pad*2])
    data_bigger[pad:-pad, pad:-pad, pad:-pad] = data
    data_out = applyTransformToVolume(data_bigger, data_bigger, transform_rotation)

    # Then only axial rotation. Can't do all axis at once to match Fiji
    if rot_z_Fiji != 0:
        center_xyz = (data_out.shape[2] / 2.0, data_out.shape[1] / 2.0, data_out.shape[0] / 2.0)
        transform_rotation.SetFixedParameters((center_xyz[0], center_xyz[1], center_xyz[2], center_angle))
        eulerAngle_xyz_deg = (0.0, 0.0, -rot_z_Fiji)
        offsets_xyz2 = (0.0, -50.0, 0.0)
        transform_rotation.SetParameters((eulerAngle_xyz_deg[0]/180.0*np.pi, eulerAngle_xyz_deg[1]/180.0*np.pi, eulerAngle_xyz_deg[2]/180.0*np.pi, offsets_xyz2[0], offsets_xyz2[1], offsets_xyz2[2]))
        data_out2 = applyTransformToVolume(data_out, data_out, transform_rotation)
    else:
        data_out2 = data_out

    return(data_out2[pad:-pad, pad:-pad, pad:-pad])

def analyze_pores_2d(this_scan, mask_pores):
    tic = time.time()

    this_scan.slice_angles_to_growth = []
    this_scan.slice_ellipse_major_diameters = []
    this_scan.slice_ellipse_minor_diameters = []
    this_scan.slice_ellipse_eccentricity = []
    this_scan.slice_areas = []
    this_scan.slice_eq_diameters = []
    this_scan.slice_perimeters = []

    for ind, slice in enumerate(mask_pores):
        slice_lab = measure.label(slice)
        slice_properties = np.array(measure.regionprops(slice_lab))

         # Angle w.r.t vertical (growth axis), with positive being CCW, in degrees, from -90 to 90
        angles = 180 / np.pi * np.array([p.orientation for p in slice_properties]) # From -90 to 90
        # angles[angles<0] = 180+angles[angles<0]
        this_scan.slice_angles_to_growth.append(angles)

        this_scan.slice_ellipse_major_diameters.append(np.array([p.axis_major_length for p in slice_properties]))
        this_scan.slice_ellipse_minor_diameters.append(np.array([p.axis_minor_length for p in slice_properties]))
        this_scan.slice_ellipse_eccentricity.append(np.array([p.eccentricity for p in slice_properties]))
        this_scan.slice_eq_diameters.append(np.array([p.equivalent_diameter_area for p in slice_properties]))
        this_scan.slice_perimeters.append(np.array([p.perimeter for p in slice_properties]))
        this_scan.slice_areas.append(np.array([p.area for p in slice_properties]))

        if ind % 100 == 0:
            print('.', end='')

    return(this_scan)

def mean_attenuation_volume(this_scan, mask_to_use, data):
    mu_s = np.nanmean(data[mask_to_use])
    V_s = np.nansum(mask_to_use.flatten())
    V_s_m = np.nansum(mask_to_use.flatten())*(voxel_size*10**-6)**3 #[m3]
    return(mu_s,V_s, V_s_m)

def cleanup_keep_single_particle(mask_solid):

    # Morphological opening to separate sample from artifact regions
    chunk_size = (500, 500, 500)
    filter_half_width = 7
    structure = custom_3d_kernel_sphere(filter_half_width)
    mask_solid_eroded = ~apply_3d_image_processing_on_subvolumes(~mask_solid, fast_pytorch_mask_dilation, chunk_size_max = chunk_size, overlap = filter_half_width, radius = filter_half_width)
    mask_solid_opened = apply_3d_image_processing_on_subvolumes(mask_solid_eroded, fast_pytorch_mask_dilation, chunk_size_max = chunk_size, overlap = filter_half_width, radius = filter_half_width)

    # Isolate largest connected particle
    lab = measure.label(mask_solid_opened)
    properties = np.array(measure.regionprops(lab))
    areas = np.array([e.area for e in properties])
    ind_max_area = np.argmax(areas)
    mask_out = lab == properties[ind_max_area].label

    return(mask_out)

def compute_mask_solid(this_scan, data, threshold, force_redo = False):

    if force_redo or not 'b_movingRegisteredToStatic_maskSolid' in os.listdir(this_scan.full_path):

        threshold_needle = 4.0 # Linear attenuation threshold [/cm]
        mask_growth_needle = 10 # Grow the mask obtained using threshold_needle with this amount of pixels
        mask_enclosing_circle = tifffile.imread(path_save+case+'/maskEnclosingCircle.tif') > 0
        bin_factor_dilatation = 1 # Binning before smoothing, as an option to speed things up for large kernels
        filter_half_width = 3 # After binning (equal to sigma for gaussian filter)

        kernel = custom_3d_gaussian_filter(filter_half_width)

        if bin_factor_dilatation > 1:
            data_smoothed = fast_pytorch_bin_3d(data,bin_factor_dilatation, chunk_size = 71)
        else:
            data_smoothed = data

        print('\t3D smoothing for thresholding of mask_solid and mask_needle:')
        data_smoothed = apply_3d_image_processing_on_subvolumes(data_smoothed, fast_pytorch_convolution, kernel_array = kernel, chunk_size_max = (500, 500, 500), overlap = 3*filter_half_width)

        if bin_factor_dilatation > 1:
            data_smoothed = skimage.transform.rescale(data_smoothed, bin_factor_dilatation, multichannel=False)

        # File(this_scan.full_path+'/b_movingRegisteredToStatic_smoothed').saveTiffStack(data_smoothed)

        mask_needle = data_smoothed > threshold_needle
        mask_needle = apply_3d_image_processing_on_subvolumes(mask_needle, fast_pytorch_mask_dilation, chunk_size_max = (500, 500, 500), overlap = mask_growth_needle, radius = mask_growth_needle)

        mask_solid = data_smoothed > threshold
        mask_solid[mask_needle] = False

        for ind, slice in enumerate(mask_solid):
            slice[~mask_enclosing_circle] = False
            mask_solid[ind] = slice

        # Computing top and bottom indices for a ROI within the solid
        this_scan = find_top_bottom_slices_ind(this_scan, mask_solid, this_scan.ind_all_top, this_scan.ind_all_bottom)

        # Close mask_solid, using 2D fill holes algorithm slice by slice
        print('\tClosing mask solid')
        mask_solid_closed = close_mask_solid(this_scan, mask_solid, mask_needle, force_redo = force_redo)

        # Clean up mask_solid_closed and mask_solid to keep single large particle region, and remove artifacts above and below sample
        print('\tCleaning up mask solid (3D opening), isolating largest particle')
        mask_solid_closed = cleanup_keep_single_particle(mask_solid_closed)
        mask_solid_closed = mask_crop_ROI(mask_solid_closed, this_scan.ind_top-this_scan.ind_margin-20, this_scan.ind_bottom+this_scan.ind_margin+20)
        mask_solid = mask_solid * mask_solid_closed

        print('\tSaving mask_solid and mask_solid_closed:')
        File(this_scan.full_path+'/b_movingRegisteredToStatic_maskSolid').saveTiffStack(mask_solid, type = 'bool')
        File(this_scan.full_path+'/b_movingRegisteredToStatic_maskSolidClosed').saveTiffStack(mask_solid_closed, type = 'bool')
        # File(this_scan.full_path+'/b_movingRegisteredToStatic_maskNeedle').saveTiffStack(mask_needle, type = 'bool')

    else:
        print('\tRe-loading mask_solid and mask_solid_closed')
        mask_solid = File(this_scan.full_path+'/b_movingRegisteredToStatic_maskSolid').readAll()
        mask_solid_closed = File(this_scan.full_path+'/b_movingRegisteredToStatic_maskSolidClosed').readAll()
        # mask_needle = File(this_scan.full_path+'/b_movingRegisteredToStatic_maskNeedle').readAll()

    return(this_scan, mask_solid.astype('bool'), mask_solid_closed.astype('bool'))

In [13]:
# Rotation
rot_coronal_Fiji = 1.0 # [Deg]
rot_sagittal_Fiji = -1.0 # [Deg]
rot_z_Fiji = 43.0 # [Deg] This angle: such that tree growth is vertical from bottom to top in Fiji, after sagittal / coronal rotation

# Top and bottom indices along z that contain sample for all scans (after rotation /!\)
ind_all_top = 125
ind_all_bottom = 510

threshold = 0.10 # Linear attenuation threshold [/cm], gas to wood

In [8]:
# Rotation
rot_coronal_Fiji = -1.4 # [Deg]
rot_sagittal_Fiji = -5.0 # [Deg]
rot_z_Fiji = -169.0 # [Deg] This angle: such that tree growth is vertical from bottom to top in Fiji, after sagittal / coronal rotation

# Top and bottom indices along z that contain sample for all scans (after rotation /!\)
ind_all_top = 140
ind_all_bottom = 470

threshold = 0.065 # Linear attenuation threshold [/cm], gas to wood

In [8]:
# Rotation
rot_coronal_Fiji = -1.0 # [Deg]
rot_sagittal_Fiji = -5.0 # [Deg]
rot_z_Fiji = -91.5 # [Deg] This angle: such that tree growth is vertical from bottom to top in Fiji, after sagittal / coronal rotation

# Top and bottom indices along z that contain sample for all scans (after rotation /!\)
ind_all_top = 130
ind_all_bottom = 465

threshold = 0.05 # Linear attenuation threshold [/cm], gas to wood

In [11]:
# Rotation
rot_coronal_Fiji = -0.2 # [Deg]
rot_sagittal_Fiji = 0.5 # [Deg]
rot_z_Fiji = 40.0 # [Deg] This angle: such that tree growth is vertical from bottom to top in Fiji, after sagittal / coronal rotation

# Top and bottom indices along z that contain sample for all scans (after rotation /!\)
ind_all_top = 170
ind_all_bottom = 480

threshold = 0.05 # Linear attenuation threshold [/cm], gas to wood

In [9]:
ind_scan = 26
this_scan = list_scans[ind_scan]
print('Running scan #'+str(ind_scan+1)+' - '+this_scan.folder_name)

# Re-load scan data
with open(path_save+case+'/post_pro_data.pickle', 'rb') as handle:
    list_scans = pickle.load(handle)

torch.cuda.set_device(0)
print(torch.cuda.current_device())

Running scan #27 - 16b
0


In [14]:
##%% Main loop: run over all scans

# Re-load scan data
with open(path_save+case+'/post_pro_data.pickle', 'rb') as handle:
    list_scans = pickle.load(handle)

nb_scans = len(list_scans)
force_redo = True

for ind_scan,this_scan in enumerate(list_scans[26:]):
    tic_scan = time.time()

    print('Running scan #'+str(ind_scan+1)+'/'+str(nb_scans)+' - '+this_scan.folder_name)

    # Loading data
    tic = time.time()
    data = File(path_save+case+'/'+this_scan.folder_name+'/b_movingRegisteredToStatic/').readAll()
    print('\tLoading data took: '+str(time.time()-tic))

    # Rotating data
    tic = time.time()
    data = rotate_data(data, rot_coronal_Fiji, rot_sagittal_Fiji, rot_z_Fiji)
    print('\tRotating data took: '+str(time.time()-tic))
    File(path_save+case+'/'+this_scan.folder_name+'/b_movingRegisteredToStatic_rotatedZ/').saveTiff(data[data.shape[0]//2-100,:,:], ind=0)
    File(path_save+case+'/'+this_scan.folder_name+'/b_movingRegisteredToStatic_rotatedZ/').saveTiff(data[data.shape[0]//2,:,:], ind=1)
    File(path_save+case+'/'+this_scan.folder_name+'/b_movingRegisteredToStatic_rotatedZ/').saveTiff(data[data.shape[0]//2+100,:,:], ind=2)
    File(path_save+case+'/'+this_scan.folder_name+'/b_movingRegisteredToStatic_rotatedXY/').saveTiff(data[:,data.shape[1]//2,:], ind=0)
    File(path_save+case+'/'+this_scan.folder_name+'/b_movingRegisteredToStatic_rotatedXY/').saveTiff(data[:,:,data.shape[2]//2], ind=1)

    # Computing/Loading masks solid
    this_scan.ind_all_top = ind_all_top
    this_scan.ind_all_bottom = ind_all_bottom
    this_scan, mask_solid, mask_solid_closed = compute_mask_solid(this_scan, data, threshold, force_redo = force_redo)

    # Compute mean solid attenuation and solid volume before cropping
    this_scan.mu_solid_not_cropped_open, this_scan.volume_solid_not_cropped_open, this_scan.volume_solid_not_cropped_open_m3 = mean_attenuation_volume(this_scan, mask_solid, data)
    this_scan.mu_solid_not_cropped_closed, this_scan.volume_solid_not_cropped_closed, this_scan.volume_solid_not_cropped_closed_m3 = mean_attenuation_volume(this_scan, mask_solid_closed, data)
    this_scan.porosity_not_cropped = 1 - np.sum(mask_solid.flatten()) / np.sum(mask_solid_closed.flatten())

    # Crop ROI in mask_solid and mask_solid_closed
    mask_solid = mask_crop_ROI(mask_solid, this_scan.ind_top, this_scan.ind_bottom)
    mask_solid_closed = mask_crop_ROI(mask_solid_closed, this_scan.ind_top, this_scan.ind_bottom)

    # Compute mean solid attenuation and solid volume after cropping
    this_scan.mu_solid_open, this_scan.volume_solid_open, this_scan.volume_solid_open_m3 = mean_attenuation_volume(this_scan, mask_solid, data)
    this_scan.mu_solid_closed, this_scan.volume_solid_closed, this_scan.volume_solid_closed_m3 = mean_attenuation_volume(this_scan, mask_solid_closed, data)
    this_scan.porosity = 1 - np.sum(mask_solid.flatten()) / np.sum(mask_solid_closed.flatten())

    # Analyze pore structure (3D)
    mask_pores = mask_solid_closed * (~mask_solid)
    tic = time.time()
    lab = measure.label(mask_pores)
    properties = np.array(measure.regionprops(lab))
    print('\tParticle identification and analysis took: '+str(time.time()-tic))

    # Cleaning up pores (3D)
    properties = compute_height(properties)
    areas = np.array([prop.area for prop in properties])
    height = np.array([prop.height for prop in properties])
    this_scan.ind_properties_cleaned = np.where((areas>500) * (areas < 300000) * (height > 100))[0]
    properties_cleaned = properties[this_scan.ind_properties_cleaned]

    # Compute equivalent cylinder diameters (3D)
    properties_cleaned = compute_equivalent_cylinder_diameters(properties_cleaned)
    this_scan.nb_pores = len(properties_cleaned)
    # this_scan.pore_properties = properties_cleaned # Don't save this, otherwise pickle too heavy (image data stored)

    # Analyze pore structure (2D, slice by slice)
    print('\tStarting 2D pore analysis ', end ='')
    tic = time.time()
    this_scan = analyze_pores_2d(this_scan, mask_pores)
    print(' Done, it took: '+str(time.time()-tic))

    # Build and analyze skeleton for tortuosity
    tic = time.time()
    skeleton = compute_skeleton(this_scan, mask_pores, force_redo = force_redo)
    skeleton_lab = measure.label(skeleton)
    skeleton_properties = measure.regionprops(skeleton_lab)
    print('\tSkeletonize and analysis took: '+str(time.time()-tic))

    # Extract which pores to keep from skeleton
    skeleton_properties = compute_height(skeleton_properties)
    skeleton_areas = np.array([prop.area for prop in skeleton_properties])
    skeleton_height = np.array([prop.height for prop in skeleton_properties])
    this_scan.skeleton_ind_properties_cleaned = np.where((skeleton_areas>50) * (skeleton_areas < 1000) * (skeleton_height > 100))[0]

    # Compute tortuosity for each pore
    skeleton_properties_cleaned = compute_tortuosity(np.array(skeleton_properties)[this_scan.skeleton_ind_properties_cleaned])

    # Save relevant pore & skeleton properties
    this_scan.areas = [prop.area for prop in properties_cleaned]
    this_scan.equivalent_cylinder_diameters = [prop.equivalent_cylinder_diameter for prop in properties_cleaned]
    this_scan.heights = [prop.height for prop in properties_cleaned]
    this_scan.skeleton_pore_path_length = [prop.length_skeleton for prop in skeleton_properties_cleaned]
    this_scan.tortuosity = [prop.tortuosity for prop in skeleton_properties_cleaned if not np.isnan(prop.tortuosity) and prop.tortuosity > 1]

    # Update save of scan data
    print('\tUpdating save pickle')
    with open(path_save+case+'/post_pro_data.pickle', 'wb') as handle:
        pickle.dump(list_scans, handle)

    print('\tThis scan took: '+str(time.time()-tic_scan))
print('Done')


Running scan #1/40 - 16b
	Loading data took: 23.302690505981445
	Rotating data took: 10.08382248878479
	3D smoothing for thresholding of mask_solid and mask_needle:
	 Processing 18 chunks of size [349 426 426] .................. Done
	 Processing 18 chunks of size [349 426 426] .................. Done
		Detected ROI:	ind_top: 165, ind_bottom: 450, height: 286
	Closing mask solid
	Cleaning up mask solid (3D opening), isolating largest particle
	 Processing 18 chunks of size [349 426 426] .................. Done
	 Processing 18 chunks of size [349 426 426] .................. Done
	Saving mask_solid and mask_solid_closed:
	Particle identification and analysis took: 10.445164442062378
	Starting 2D pore analysis ....... Done, it took: 44.405805826187134
	Skeletonize and analysis took: 63.365785360336304
	Updating save pickle
	This scan took: 321.6662292480469
Running scan #2/40 - 17a
	Loading data took: 27.724085807800293
	Rotating data took: 10.212224006652832
	3D smoothing for thresholdin

In [59]:
with open(path_save+case+'/post_pro_data.pickle', 'rb') as handle:
    list_scans = pickle.load(handle)

In [10]:
print(len(list_scans))

ind_scan = 34

print(list_scans[ind_scan].ind)
print(list_scans[ind_scan].folder_name)
print(list_scans[ind_scan].porosity)
print(list_scans[ind_scan].nb_pores)


40
34
20b
0.21614414068577625
9


In [17]:
ind_slice_X = 600-1 # Reslice from Top in Fiji
ind_slice_Y = 620-1 # Reslice from Left in Fiji
nb_scans = len(list_scans)

for ind_scan,this_scan in enumerate(list_scans[-2:]):

    print('Loading scan #'+str(ind_scan+1)+'/'+str(nb_scans)+' - '+this_scan.folder_name)

    # Loading data
    tic = time.time()
    data = File(path_save+case+'/'+this_scan.folder_name+'/b_movingRegisteredToStatic/').readAll()
    print('\tLoading data took: '+str(time.time()-tic))

    File(path_save+case+'/probe_sliceX_'+str(ind_slice_X).zfill(4)).saveTiff(data[:,ind_slice_X,:], ind = this_scan.ind)
    File(path_save+case+'/probe_sliceY_'+str(ind_slice_Y).zfill(4)).saveTiff(data[:,:,ind_slice_Y], ind = this_scan.ind)

	Loading data took: 37.67014169692993
Loading scan #1/41 - 23


In [11]:
print(time.time())

1658937477.8635976


In [12]:
for scan in list_scans[:]:
    scan.voxel_size = voxel_size

    # 2D slice properties
    scan.slice_ind_keep = np.concatenate(scan.slice_eq_diameters) > 2
    scan.slice_mean_eq_diameter = np.nanmean(np.concatenate(scan.slice_eq_diameters)[scan.slice_ind_keep]) * voxel_size
    scan.slice_mean_eccentricity = np.nanmean(np.concatenate(scan.slice_ellipse_eccentricity)[scan.slice_ind_keep])
    scan.slice_mean_angle_to_growth = np.nanmean(np.concatenate(scan.slice_angles_to_growth)[scan.slice_ind_keep])

    # 3D pore/skeleton properties
    scan.mean_eq_diameter = np.nanmean(scan.equivalent_cylinder_diameters)
    scan.mean_tortuosity = np.nanmean(scan.tortuosity)

    # 3D single particle properties, assuming a cylindrical shape
    scan.total_sample_height = scan.ind_bottom+scan.ind_margin - (scan.ind_top-scan.ind_margin)
    scan.total_sample_height_microns = scan.total_sample_height * voxel_size
    scan.total_sample_eq_diameter = np.sqrt(4.0 / np.pi * scan.volume_solid_not_cropped_closed / scan.total_sample_height)
    scan.total_sample_eq_diameter_microns = scan.total_sample_eq_diameter * voxel_size

    print(scan.folder_name)
print('Done')

01
02
03
04a
04b
05a
05b
06a
06b
07a
07b
08a
08b
09a
09b
10a
10b
11
12
13a
13b
14a
14b
15a
15b
16a
16b
17a
17b
18a
18b
19a
19b
20a
20b
21a
21b
22
23
24
Done


/global/homes/e/eboigne/local/miniconda3/envs/als/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice
  if sys.path[0] == "":


In [13]:
print('\tUpdating save pickle')
with open(path_save+case+'/post_pro_data.pickle', 'wb') as handle:
    pickle.dump(list_scans, handle)


	Updating save pickle


In [20]:
print(list_scans[0].slice_)



12.143631982946165
